# Introduction
The python web api implementation found in the brevettai package gives a lightweight programmatical interface to all the functionality that the [Brevetti AI platform](https://platform.brevetti.ai) offers.

This enables high level access for
* automation of tasks on the platform
* tagging of datasets or models
* dataset management
* managing models etc...

This document shows how this api can be used to get access to datasets and create a model training job.

Web access is granted with your website user, allowing you to automate tasks on the platform. In Python this is achieved through the **BrevettiAI** object.

# Brevetti AI package installation and imports
Install brevettiai using the pip package manager.

In [ ]:
pip install brevettiai

In [ ]:
# Setup logging to avoid verbosity
import logging
log = logging.getLogger(__name__)
logging.basicConfig()
log.root.setLevel(logging.DEBUG)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("tensorflow").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

# API: BrevettiAI Login

High level access for automation of tasks on the platform, tagging, dataset management, models, etc...

## Platform Login
As on the web page you have 60 minutes of access before needing to log back in.

In [ ]:
# Imports and setup
from brevettiai.platform import BrevettiAI

web = BrevettiAI()

help(web)

In [ ]:
?web

# API: Element acces, list datasets, tags, models...

with the web object you can list, manage and edit elements on the web page.
Most of the functions require an id, the guid of the object to manipulate. Alternatively to get all use None as id.

EX: to list datasets, tags, and models, call get_... with no id (id=None)

In [ ]:
datasets = web.get_dataset()
tags = web.get_tag()
models = web.get_model()

# List 10 first dataset names
[d.name for d in datasets][:10]

For a single dataset, model or ... use the get_... functions with an id

In [ ]:
dataset = web.get_dataset(datasets[0].id)
dataset

# API: Customized Job Settings
Settings are esstentially the serialized configuration of a training job algorithm.
Settings can be used for a training job configuration by letting a user change settings, and settings are included in the default job output, such that the parameters of a training job can be saved and tracked for comparison and audit purposes.

In [ ]:
from brevettiai import Job, JobSettings
        
class MyAlgoObject(JobSettings):
    multiply_factor: float = 2.0
    enable : bool = True

    def __call__(self, x):
        factor = 1.0
        if self.enable:
            factor *= self.multiply_factor
        return x * factor
test_obj = MyAlgoObject(multiply_factor=3.0)

# Settings used for creating the job
print(test_obj, test_obj(2))

In [ ]:
class MyJob(Job):
    settings: MyAlgoObject
    
    def run(self): # This function should be overloaded and is run when job is started

        print(f"Run my custom code using custom parameters : {self.settings.__dict__}")
        print(f"Result on input 2.0: {self.settings(2.0)}")
        return None # Return path to model artifacts to be uploaded after job is completed
        


# API: Create Model Training Job
To enter the job context you can either create a model on the platform or programatically via the web api.

The following code finds the firs dataset and creates a model (job) with access to this model.
The model context type is the id of a model type on the platform to use.
After running the model is available on the website, along with an s3 bucket for artifacts for your job outputs


When creating a model you have the option to include datasets and tags and settings defining your model.

In [ ]:
# Datasets to add to the created job
datasets = web.get_dataset()[1:2]

model = web.create_model(name=f'Test {web.user["firstName"]} {web.user["lastName"]}',
                         settings=test_obj,
                         datasets=datasets)

## Start job

The model id and the model api key gives you access to use the python sdk to access data, and to upload artifacts and lots of other cool stuff. To enable this, we need to start model training - this is the same as selecting "Simulate training" on the platform.

In [ ]:
# Starting training in simulate mode
job = web.initialize_training(model=model, job_type=MyJob)
print(f"Model url: {web.host}/models/{model.id} (Please check it out :)\n")
print("To access data and model through python SDK use the following")
print(f"Model id: {model.id}")
print(f"Model api key: {model.api_key}")

In [ ]:
job.start()

## NB: Delete job
If the job has not been deployed, and you are e.g. just testing interfaces, you may delete a job

In [ ]:
# NB: delete model, there is no simple "undo" funcionality for this
web.delete(model)